In [1]:
!pip install spacy
!python -m spacy download en_core_web_md


/Users/chelseashe/Thought_Toolkit/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.5/33.5 MB 28.6 MB/s eta 0:00:00a 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')


### Set Up

In [2]:
import json
import os

# %pip install tqdm
from tqdm import tqdm # A process tracker

# Import a helper file containing tools
import utils

# Import parsing logic for processing logs into structured actions
import action_parser

# Import a helper file containing functions for computing outcome measures over the entire text such as semantic expansion and coordination
import outcome_measures

# Import helper files containing functions required to further parse actions with more advanced interpretations
import level_2_comparisons
import level_3_comparisons

/Users/chelseashe/Thought_Toolkit/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Logs successfully saved to: /Users/chelseashe/Thought_Toolkit/pilot1_logs.json


/Users/chelseashe/Thought_Toolkit/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Extracting Raw Logs from the CoAuthor Website

Before running the cells in this notebook, ensure you have executed the `extract_coauthor_raw_logs.py` script. This script extracts and generates raw logs along with writing session metadata into a structured JSON file from the CoAuthor website. The raw logs are saved into the `coauthor_logs_by_session.json` file.

#### Example Prompts:

For demonstration purposes, the following prompts are used in the script:

- **Argumentative Writing Prompts**:  
  `["dating", "school", "athletes"]`

- **Creative Writing Prompts**:  
  `["shapeshifter", "mattdamon", "mana"]`

#### Example Results:

Here are the counts of writing sessions for each prompt:

- **Argumentative Prompts**:  
  - `argumentative_dating`: 60 sessions  
  - `argumentative_school`: 66 sessions  
  - `argumentative_athletes`: 60 sessions  

- **Creative Prompts**:  
  - `creative_shapeshifter`: 82 sessions  
  - `creative_mattdamon`: 79 sessions  
  - `creative_mana`: 90 sessions  

- **Total**: 437 writing sessions with raw logs.

Feel free to define your own prompts in the `extract_coauthor_raw_logs.py` script to align with your objectives for generating the raw log JSON file.

---

#### How to Run:

1. Open a terminal and navigate to the folder containing the `extract_coauthor_raw_logs.py` script. You can use the `cd` (change directory) command to do this.

   **Example:**  
   ```bash
   cd ~/Desktop/"Writing Assistant Toolkit"

2. Run the script using Python.

   **Command:**
   ```bash
   python extract_coauthor_raw_logs.py

In [3]:
# #Extracting logs from zip file
# import zipfile

# folder_name = '/content/vibewritingpilot.zip'

# with zipfile.ZipFile(folder_name, 'r') as zip_ref:
#     zip_ref.extractall('/content/pilot2')  # This will extract to /content/pilot_2

In [ ]:
import os
import os
import json

def load_jsonl_files_parsed(folder_path):
    """
    Reads all .jsonl files in the given folder and returns a dictionary.

    Each key in the dictionary is the file name (without extension),
    and its value is a list of parsed JSON objects (not strings).
    """
    jsonl_data = {}

    for filename in os.listdir(folder_path):
        if filename.endswith('.jsonl'):
            full_path = os.path.join(folder_path, filename)
            try:
                with open(full_path, 'r', encoding='utf-8') as file:
                    lines = file.read().splitlines()
                    parsed_lines = [json.loads(line) for line in lines]
                    jsonl_data[filename.split('.')[0]] = parsed_lines
            except Exception as e:
                print(f"Error reading {filename}: {e}")

    return jsonl_data

# Example usage
folder_path = "pilot2_raw_logs"  # Your folder path
data = load_jsonl_files_parsed(folder_path)

# Print a preview of one file to confirm the format
for file_name, events in data.items():
    print(f"\nFile: {file_name}, Number of events: {len(events)}")
    if events:
        print("First event:")
        print(json.dumps(events[0], indent=2))  # Pretty-print first event
    break  # Remove this if you want to preview more files


File: f9da5086528a4eeab85bbf1b0f8bb279, Number of events: 2871
First event:
{
  "eventName": "system-initialize",
  "eventSource": "api",
  "eventTimestamp": 1750169276554,
  "textDelta": "",
  "cursorRange": "",
  "currentDoc": "\n",
  "currentCursor": 0,
  "currentSuggestions": [],
  "currentSuggestionIndex": 0,
  "currentHoverIndex": "",
  "currentN": "",
  "currentMaxToken": "",
  "currentTemperature": "",
  "currentTopP": "",
  "currentPresencePenalty": "",
  "currentFrequencyPenalty": "",
  "originalSuggestions": []
}


In [ ]:
jsonl_data = {}
filename = "pilot_raw_logs/f9da5086528a4eeab85bbf1b0f8bb279.jsonl"
# Print a preview of one file to confirm the format
with open(filename, 'r', encoding='utf-8') as file:
    lines = file.read().splitlines()
    parsed_lines = [json.loads(line) for line in lines]
    jsonl_data[filename.split('.')[0]] = parsed_lines

data = jsonl_data
print(data)

{'pilot1_raw_logs/f9da5086528a4eeab85bbf1b0f8bb279': [{'eventName': 'system-initialize', 'eventSource': 'api', 'eventTimestamp': 1750169276554, 'textDelta': '', 'cursorRange': '', 'currentDoc': '\n', 'currentCursor': 0, 'currentSuggestions': [], 'currentSuggestionIndex': 0, 'currentHoverIndex': '', 'currentN': '', 'currentMaxToken': '', 'currentTemperature': '', 'currentTopP': '', 'currentPresencePenalty': '', 'currentFrequencyPenalty': '', 'originalSuggestions': []}, {'eventName': 'text-insert', 'eventSource': 'user', 'eventTimestamp': 1750170154661, 'textDelta': {'ops': [{'insert': 'W'}]}, 'cursorRange': '', 'currentDoc': '', 'currentCursor': 1, 'currentSuggestions': [], 'currentSuggestionIndex': 0, 'currentHoverIndex': '', 'currentN': '5', 'currentMaxToken': '50', 'currentTemperature': '0.95', 'currentTopP': '1', 'currentPresencePenalty': '0.5', 'currentFrequencyPenalty': '0.5', 'originalSuggestions': []}, {'eventName': 'text-insert', 'eventSource': 'user', 'eventTimestamp': 1750170

In [6]:
logs_by_session = data

In [7]:
for key in logs_by_session.keys():
  logs_by_session[key] = logs_by_session[key][1:]

### Loading Structured JSON File for Parsing and Analysis

Once the `extract_coauthor_raw_logs.py` script has successfully generated a structured JSON file, you can proceed to load the file in this section for further parsing and analysis.

In [ ]:
# Use the current working directory
script_dir = os.getcwd()

# Get the raw log dataset
file_path = os.path.join(script_dir, 'pilot2_logs.json')

# Feel free to uncomment the line below and start with a smaller sample (20 writing sessions) to reduce runtime
# file_path = os.path.join(script_dir, 'small_logs_for_test.json')

# Open and load the JSON file
with open(file_path) as f:
    logs_by_session = json.load(f)

In [9]:
import os
import re
import json

"""
# Load raw JSON-like data from file
file_path = os.path.join(script_dir, '/content/testing_new_site.json')
with open(file_path, 'r') as f:
    raw_data = f.read()
"""

raw_data = logs_by_session


def fix_all_arrays(raw_json_string):
    def fix_array(match):
        array_content = match.group(1)
        # Add commas between objects: }{
        fixed = re.sub(r'\}\s*\{', '},\n{', array_content.strip())
        return f'[{fixed}]'

    # Regex: match any array content following a key
    fixed_json_string = re.sub(r'\[\s*({.*?})\s*\]', lambda m: fix_array(m), raw_json_string, flags=re.DOTALL)
    return fixed_json_string


# This regex looks for a pattern where a closing brace is immediately followed (with any whitespace) by an opening brace,
# and inserts a comma between them.
raw_data_str = json.dumps(logs_by_session)
fixed_data = re.sub(r'(\})\s*(\{)', r'\1, \2', raw_data_str)


# Now try to load the fixed data as JSON.
try:
    logs_by_session = json.loads(fixed_data)
    print("✅ Fixed and loaded JSON successfully!")
except json.JSONDecodeError as e:
    print("❌ Failed to parse JSON:", e)


for test_user, events in logs_by_session.items():
    if isinstance(events, list):
       logs_by_session[test_user] = [
            event for event in events if event.get("eventName") != "system-initialize"
        ]

✅ Fixed and loaded JSON successfully!


### Parsing Raw Log JSON File into Structured Level 1 Actions

This section processes raw logs and converts them into Level 1 actions using a analyzer. Each parsed action is enriched with a **level_1_action_type** key, which specifies the action type (e.g., `insert_text`, `delete_text`, `accept_suggestion`).

**level_1_actions_per_session** is a dictionary where each session key maps to a list of parsed actions, organizing the output by session for streamlined analysis and further processing.

In [10]:
# Use the current working directory
script_dir = os.getcwd()


In [ ]:
# Initialize an empty dictionary to store parsed actions
level_1_actions_per_session = {}

# Iterate through all sessions in the raw logs and parse actions
for session in tqdm(logs_by_session, desc="Parsing Level 1 Actions"):

    # Initialize the MergeActionsAnalyzer for each session
    actions_analyzer = action_parser.MergeActionsAnalyzer(
        last_action=None,
        raw_logs=logs_by_session[session]
    )

    # Parse the logs for the session into structured actions
    actions_lst, last_action = actions_analyzer.parse_actions_from_logs(
        all_logs=logs_by_session[session],
        last_action=None,
        DLT_CHAR_MAX_COUNT=9  # Optional: Specify tiny delete threshold here
    )

    # Store the parsed actions in the output dictionary
    level_1_actions_per_session[session] = actions_lst

# Add a new key to each action for classification and further analysis
for session_key, actions in level_1_actions_per_session.items():
    for action in actions:
        # Add 'level_1_action_type' as a duplicate of 'action_type'
        action['level_1_action_type'] = action['action_type']

Parsing Level 1 Actions: 100%|██████████| 5/5 [00:00<00:00, 42.07it/s]


In [12]:
level_1_actions_per_session.keys()

dict_keys(['legislation_corporate_1', 'legislation_corporate_2', 'legislation_antitrust_1', 'legislation_antitrust_2', 'legislation_antitrust_3'])

In [13]:
[
    (action["action_type"], action.get("action_delta"))
    for action in level_1_actions_per_session["legislation_corporate_1"]
]

[('insert_text',
  ('INSERT', 'Corporate personhood has its pros and cons. ', 44, 7)),
 ('query_suggestion', ''),
 ('present_suggestion', ''),
 ('hover_over_text', ''),
 ('accept_suggestion', ''),
 ('present_suggestion', ''),
 ('insert_text',
  ('INSERT',
   "Corporate personhood has its pros and cons. One advantage is that it allows corporations to enter contracts, sue or be sued, which promotes business activities and economic growth. However, this concept can lead to abuse of power, as some corporations might exploit these. Let's talk about the advantages first. Corporate personhood allows corporations to have basic rights, which ",
   381,
   58)),
 ('query_suggestion', ''),
 ('present_suggestion', ''),
 ('hover_over_text', ''),
 ('cursor_operation', ''),
 ('reject_suggestion', ''),
 ('delete_text',
  ('DELETE', 'Corporate personhood has its pros and cons.', 43, 7)),
 ('cursor_operation', ''),
 ('insert_text',
  ('INSERT',
   "is guaranteed in the constitution. People get rights wh

In [14]:
[
    action if action["action_type"] == "present_suggestion" else None
    for action in level_1_actions_per_session["legislation_corporate_1"]
]

[None,
 None,
 {'action_type': 'present_suggestion',
  'action_source': 'api',
  'action_logs': [{'eventName': 'suggestion-open',
    'eventSource': 'api',
    'eventTimestamp': 1750095856681,
    'textDelta': '',
    'cursorRange': '',
    'currentDoc': '',
    'currentCursor': 44,
    'currentSuggestions': [{'index': 0,
      'original': 'For instance, it allows companies the right to enter into contracts, sue or be sued, and own property as distinct legal entities.',
      'trimmed': 'For instance, it allows companies the right to enter into contracts, sue or be sued, and own property as distinct legal entities.',
      'probability': None,
      'source': 'GPT-4o'},
     {'index': 1,
      'original': 'However, understanding its implications can help us navigate the complex relationship between corporations, individuals, and society.',
      'trimmed': 'However, understanding its implications can help us navigate the complex relationship between corporations, individuals, and socie

### Save Parsed Level 1 Actions to a JSON File in the Current Directory

This section saves the **level_1_actions_per_session** dictionary into a JSON file named `level_1_actions_per_session.json`. The file will be created in the same directory as this notebook for convenient access to the parsed output.

#### Key Fields Included in Each Entry:

- **action_type**: The type of the action (e.g., `insert_text`, `delete_text`).

- **action_source**: Specifies whether the action originated from the user or the API.  

- **action_logs**: A list of raw logs corresponding to the action.  

- **action_start_log_id**: The log ID where the action starts.  

- **action_start_time**: The timestamp of when the action begins.  

- **action_end_time**: The timestamp of when the action ends.  

- **writing_modified**: A boolean indicating whether the writing was modified during the action.  

- **action_delta**: The changes made to the text as a result of the action.  

- **action_end_writing**: The final state of the text after the action is completed.  

- **action_end_mask**: The final state of the mask after the action is completed.  

- **action_modified_sentences**: A list of sentences affected by the action.  

- **sentences_temporal_order**: The order of sentences after modifications.  

By saving the parsed data, you create a reusable file that can be used for additional analysis or as input for other tools.

In [15]:
# Define the output file path in the same directory as this notebook
output_file = os.path.join(script_dir, 'level_1_actions_per_session.json')

# Write the parsed actions dictionary to the JSON file
with open(output_file, 'w') as f:
    json.dump(level_1_actions_per_session, f, default=utils.custom_serializer)

### Level 2 Parsing Based on Generated Level 1 Actions and Outcome Measures Documentation

#### Key Features:

The pipeline consists of two main steps:

1. **Level 2 Parsing**: Augments Level 1 actions with detailed Level 2 attributes.
2. **Outcome Measures Computation**: Calculates semantic expansion and coordination scores for deeper insights.

#### Threshold Options:

These thresholds can be configured at the top of the **level_2_comparisons.py** file:

- **MIN_INSERT_WORD_COUNT = 10**: Minimum word count required to classify an insertion as a major action.

- **MAJOR_INSRT_MAX_SIMILARITY = 0.9**: Maximum similarity for identifying major semantic differences in insertions.

- **MINOR_INSRT_MAX_SIMILARITY = 0.95**: Similarity threshold for minor semantic differences in insertions.  

- **MAX_SIMILARITY = 0.95**: Similarity threshold used to distinguish between major and minor deletions.

This step provides a more granular understanding of the actions performed in each session, laying the groundwork for advanced analysis and insights.

#### Generate Level 2 Actions

In [16]:
def parse_level_2_actions(level_1_actions_per_session, similarity_fcn):
    """
    Parses Level 2 actions based on Level 1 actions, but only for insert_text actions.

    Args:
        level_1_actions_per_session (dict): A dictionary where each session key maps to a list of level 1 actions.
        similarity_fcn (function): A similarity function (e.g., SpaCy) used to compare writing.

    Returns:
        dict: The updated actions dictionary with Level 2 attributes added to each action.
    """
    for session_id, actions_lst in tqdm(level_1_actions_per_session.items(), desc="Parsing Level 2 Actions"):
        prev_writing = None  # Tracks the end writing of the previous action

        for idx, action in enumerate(actions_lst):
            try:
                # Only process insert_text actions, skip deletions
                if action.get('level_1_action_type') != 'insert_text':
                    continue

                action['level_2_info'] = {}

                # Compute similarity with the previous action's writing
                if idx > 0 and 'action_end_writing' in action and prev_writing:
                    prev_writing_similarity, sents_info_dct = level_2_comparisons.get_similarity_with_prev_writing_for_level_2(
                        action, prev_writing, similarity_fcn
                    )

                    action['level_2_info']['similarity'] = prev_writing_similarity
                    action['level_2_info'].update(sents_info_dct)

                    # Determine the Level 2 action type
                    if level_2_comparisons.parse_level_2_major_insert_major_semantic_diff(action, prev_writing_similarity):
                        action['level_2_action_type'] = 'major_insert_major_semantic_diff'
                    elif level_2_comparisons.parse_level_2_major_insert_minor_semantic_diff(action, prev_writing_similarity):
                        action['level_2_action_type'] = 'major_insert_minor_semantic_diff'
                    elif level_2_comparisons.parse_level_2_minor_insert_major_semantic_diff(action, prev_writing_similarity):
                        action['level_2_action_type'] = 'minor_insert_major_semantic_diff'
                    elif level_2_comparisons.parse_level_2_minor_insert_minor_semantic_diff(action, prev_writing_similarity):
                        action['level_2_action_type'] = 'minor_insert_minor_semantic_diff'

                # Update the previous writing for the next iteration
                prev_writing = action.get('action_end_writing', prev_writing)

            except Exception as e:
                print(f"There is an error in session {session_id}, action index {idx}: {e}")
                continue

    return level_1_actions_per_session

# Parse Level 2 actions
level_2_actions_per_session = parse_level_2_actions(
    level_1_actions_per_session,
    similarity_fcn=utils.get_spacy_similarity
)

Parsing Level 2 Actions:  60%|██████    | 3/5 [00:01<00:00,  2.10it/s]/Users/chelseashe/Thought_Toolkit/utils.py:53: UserWarning: [W008] Evaluating Doc.similarity based on empty vectors.
  return doc1.similarity(doc2)
Parsing Level 2 Actions: 100%|██████████| 5/5 [00:02<00:00,  1.86it/s]


#### Compute Outcome Measures

In [17]:
# Compute outcome measures by iterating through each writing session after generating level 2 action types
for _, actions_lst in tqdm(level_2_actions_per_session.items(), desc="Computing Outcome Measures"):

    ##### Outcome measures applied to individual actions #####
    # Compute semantic expansion for insertion-related actions to assess textual modifications
    outcome_measures.compute_semantic_expansion(actions_lst)
    ##### End of individual action-based outcome measures #####

    ##### Outcome measures applied to the entire list of actions #####
    # Compute cumulative semantic expansion by aggregating individual expansion scores
    outcome_measures.compute_cumulative_expansion(actions_lst)

    # Evaluate coordination between AI-generated and human-inserted text,
    # capturing AI-to-human and human-to-AI interaction patterns
    outcome_measures.compute_coordination(actions_lst, similarity_fcn=utils.get_spacy_similarity)
    ##### End of list-wide outcome measures #####

Computing Outcome Measures: 100%|██████████| 5/5 [00:10<00:00,  2.11s/it]


In [18]:
[
    (
        action["action_type"],
        action.get("level_2_action_type"),
        action.get("action_delta"),
    )
    for action in level_2_actions_per_session["legislation_corporate_2"]
]

[('insert_text',
  None,
  ('INSERT', 'what are some arguments for corporate personhood', 48, 7)),
 ('query_suggestion', None, ''),
 ('present_suggestion', None, ''),
 ('hover_over_text', None, ''),
 ('insert_text', 'minor_insert_major_semantic_diff', ('INSERT', '\n', 1, 0)),
 ('reject_suggestion', None, ''),
 ('insert_text',
  'minor_insert_minor_semantic_diff',
  ('INSERT', '\nCorporate personhood ', 22, 2)),
 ('delete_text', None, ('DELETE', '\nCorporate personhood ', 22, 2)),
 ('insert_text',
  'major_insert_major_semantic_diff',
  ('INSERT',
   'Corporate personhood -- the concept that corporations can be treated as singular entities separate from any body subsumed within them -- is, to me, an illogical opinion that has done much harm ',
   193,
   32)),
 ('delete_text', None, ('DELETE', ' much harm ', 11, 2)),
 ('insert_text',
  'major_insert_minor_semantic_diff',
  ('INSERT',
   'harm to the United States and its democratic system. \n\nOne of the most significant harms done by c

In [19]:
counts = {}
for user in level_2_actions_per_session.keys():
  for action in level_2_actions_per_session[user]:
    if action.get('level_2_action_type'):
      if counts.get(action.get('level_2_action_type')):
        counts[action.get('level_2_action_type')] = counts[action.get('level_2_action_type')] + 1
      else:
        counts[action.get('level_2_action_type')] = 1

counts

{'major_insert_major_semantic_diff': 30,
 'major_insert_minor_semantic_diff': 35,
 'minor_insert_major_semantic_diff': 21,
 'minor_insert_minor_semantic_diff': 23}

### Save Parsed Level 2 Actions to a JSON File in the Current Directory

This section saves the **level_2_actions_per_session** dictionary into a JSON file named `level_2_actions_per_session.json`. The file will be created in the same directory as this notebook for convenient access to the enhanced parsed output.

#### Key Fields Included in Each Entry:

- **level_2_action_type**: The classified action type based on semantic differences (e.g., `major_insert_major_semantic_diff`).  

- **level_2_info**: Contains similarity metrics and additional details about the action:  
  - **similarity**: The similarity score between the current and previous writing.  
  - **sents_info_dct**: Additional information about sentence-level differences.  

- **action_semantic_expansion**: The amount of semantic content added or removed by the action.  

- **cumulative_semantic_expansion**: Tracks the running total of semantic expansion across the session.  

- **coordination_score**: Measures the consistency and alignment with previous actions performed by humans and AI.  

This step organizes and enriches the Level 2 parsed data, making it ready for further analysis or visualization.

In [20]:
# Define the output file path
output_file = os.path.join(script_dir, 'level_2_actions_per_session.json')

# Save the parsed actions to the JSON file
with open(output_file, 'w') as f:
    json.dump(level_2_actions_per_session, f, default=utils.custom_serializer)

# Confirm successful save
print(f"Level 2 actions successfully saved to: {output_file}")

Level 2 actions successfully saved to: /Users/chelseashe/Thought_Toolkit/level_2_actions_per_session.json


### Level 3 Parsing Based on Generated Level 2 Actions

This function processes the **level_2_actions_per_session** dictionary and augments actions with Level 3 attributes, enabling more sophisticated analysis. Key enhancements include:

#### Key Features:

- **Tracking Accepted Suggestions**: Monitors the latest accepted AI suggestions to align user insertions with AI-generated content.  

- **Identifying Mindless Patterns**: Detects specific patterns for better insight into user behavior:  
  - **Mindless Echo**: Direct repetition of AI suggestions.  
  - **Mindless Edit**: Minor modifications to AI suggestions without meaningful changes.  

- **Detecting Topic Shifts**: Identifies the introduction of new ideas and maintains a running count of detected topic shifts.  

#### Threshold Options:

These thresholds can be configured at the top of the **level_3_comparisons.py** file:

- **MIN_INSERT_WORD_COUNT = 10**: Minimum word count required for echo detection.  

- **MAX_INSERT_WORD_COUNT = 3**: Maximum word count to classify as a mindless edit.  

- **MAX_SIMILARITY_ECHO = 0.93**: Maximum similarity threshold for mindless echo classification.  

- **MAX_SIMILARITY_MINDLESS_EDIT = 0.9**: Similarity threshold for detecting mindless edits.  

- **IDEA_ALIGNMENT_MAX_SIMILARITY = 0.6**: Maximum similarity threshold for idea alignment.  

- **IDEA_ALIGNMENT_MIN_WORD_COUNT = 10**: Minimum word count threshold for detecting new ideas.  

This step enhances the parsed data with advanced attributes, providing deeper insights into user interaction patterns and their alignment with AI suggestions.

In [21]:
def parse_level_3_actions(level_2_actions_per_session, similarity_fcn):
    """
    Processes level 2 actions and enhances them with level 3 attributes:
    - Tracks the latest accepted AI suggestions for alignment with user insertions.
    - Identifies patterns like mindless echoes or edits of AI suggestions.
    - Detects topic shifts and counts new ideas introduced by the user.

    Args:
        level_2_actions_per_session (dict): A dictionary where each session key maps to a list of level 2 actions.
        similarity_fcn (function): A similarity function (e.g., SpaCy) used to compare writing.

    Returns:
        dict: The updated actions dictionary with level 3 attributes added to each action.
    """
    for _, actions_lst in tqdm(level_2_actions_per_session.items(), desc="Parsing Level 3 Actions"):
        latest_accepted_suggestion = ''  # Tracks the last accepted suggestion
        curr_idea_sentence_list = ['']  # Tracks current idea alignment sentences
        running_idea_count = 0  # Tracks the count of new ideas

        for action in actions_lst:
            try:
                action_parsed = False

                # Track the latest accepted suggestion
                if action.get('level_1_action_type') == 'insert_suggestion':
                    latest_accepted_suggestion = action.get('action_delta')[1]
                    curr_idea_sentence_list = level_3_comparisons.get_idea_alignment_order_on_AI(action, curr_idea_sentence_list, similarity_fcn)
                    if len(curr_idea_sentence_list) == 1:  # New idea detected
                        running_idea_count += 1
                    action['topic_shift'] = running_idea_count

                # Process 'insert_text' when strings are not empty or whitespaces
                if action.get('level_1_action_type') == 'insert_text' and action.get('action_delta') and \
                        action['action_delta'][1].strip():
                    # Check for major_insert_mindless_echo
                    echo_bool, echo_similarity, echo_details = level_3_comparisons.get_mindless_echo_after_AI(
                        action, latest_accepted_suggestion, similarity_fcn
                    )
                    if echo_similarity is not None:
                        action['level_3_info'] = {**echo_details, 'similarity': echo_similarity}
                    if echo_bool:
                        action['level_3_action_type'] = 'major_insert_mindless_echo'
                        action_parsed = True

                    # Check for minor_insert_mindless_edit if not already parsed
                    if not action_parsed:
                        edit_bool, edit_similarity, edit_details = level_3_comparisons.get_mindless_edit_of_AI(
                            action, latest_accepted_suggestion, similarity_fcn
                        )
                        if edit_similarity is not None:
                            action['level_3_info'] = {**edit_details, 'similarity': edit_similarity}
                        if edit_bool:
                            action['level_3_action_type'] = 'minor_insert_mindless_edit'
                            action_parsed = True

                    # Handle topic shifts for text insertions
                    if not action_parsed:
                        if action['action_delta'][-1] >= level_3_comparisons.IDEA_ALIGNMENT_MIN_WORD_COUNT:
                            curr_idea_sentence_list = level_3_comparisons.get_idea_alignment_order_on_AI(action, curr_idea_sentence_list, similarity_fcn)
                            if len(curr_idea_sentence_list) == 1:  # New idea detected
                                running_idea_count += 1
                            action['running_idea_count'] = running_idea_count
                            if running_idea_count > 0:
                                action['level_3_action_type'] = 'topic_shifted'
                        else:
                            curr_idea_sentence_list, new_idea = level_3_comparisons.get_idea_alignment_order_on_minor_insert(
                                action, curr_idea_sentence_list, similarity_fcn
                            )
                            if new_idea:
                                running_idea_count += 1
                            action['running_idea_count'] = running_idea_count
                            if running_idea_count > 0:
                                action['level_3_action_type'] = 'topic_shifted'
            except Exception as e:
                print(f"There is an error in action '{action}': {e}")
                continue

    return level_2_actions_per_session

# Parse Level 3 actions and compute additional metrics
level_3_actions_per_session = parse_level_3_actions(
    level_2_actions_per_session,
    similarity_fcn=utils.get_spacy_similarity
)

Parsing Level 3 Actions: 100%|██████████| 5/5 [00:12<00:00,  2.45s/it]


### Save Parsed Level 3 Actions to a JSON File in the Current Directory

This section saves the **level_3_actions_per_session** dictionary into a JSON file named `level_3_actions_per_session.json`. The file will be created in the same directory as this notebook, providing convenient access to the advanced parsed output.

#### Key Fields Included in Each Entry:

- **level_3_action_type**: Categorized writing behavior type (e.g., major_insert_mindless_echo or topic_shifted).

- **level_3_info**: Contains detailed analysis with similarity scores for deeper insights into user behavior.  

- **running_idea_count**: Tracks the cumulative count of detected topic shifts or new ideas introduced during the session.

This step finalizes the parsing process by adding the most advanced attributes, enabling comprehensive analysis and visualization of user interactions.

In [22]:
# Define the output file path
output_file = os.path.join(script_dir, 'level_3_actions_per_session.json')

# Save the parsed actions to the JSON file
with open(output_file, 'w') as f:
    json.dump(level_3_actions_per_session, f, default=utils.custom_serializer)

# Confirm successful save
print(f"Level 3 actions successfully saved to: {output_file}")

Level 3 actions successfully saved to: /Users/chelseashe/Thought_Toolkit/level_3_actions_per_session.json


### Assigning Action Types with Default or Custom Priority Logic

This step determines and assigns the final `action_type` to actions in the dataset using either the default priority logic or a customizable priority list.

#### Key Functions:

1. **`determine_final_action_type_default`**
   - This function automatically determines the `action_type` for each action based on the following default priority logic:
     - **Level 3 Action Types**: Highest priority.
     - **Level 2 Action Types**: Second priority.
     - **Level 1 Action Types**: Default if neither Level 3 nor Level 2 types are present.

2. **`determine_final_action_type_customize`**
   - This function allows you to use a custom priority list to determine the `action_type`.
   - By passing a predefined `priority_list` (in descending order of priority) based on your research needs, you can ensure that actions are classified according to your specific priorities.

This process provides a systematic and customizable way to ensure your action classifications are meaningful and consistent, enhancing the quality of subsequent analysis.

In [23]:
def determine_final_action_type_default(actions_dict):
    """
    Determines the final action type for each action in the actions dictionary.

    Default logic:
    - Level 3 action types have the highest priority and are considered first.
    - If no level 3 action type exists, the level 2 action type is used as the next priority.
    - If neither level 3 nor level 2 action types exist, the initial level 1 action type is retained.

    Args:
        actions_dict (dict): Dictionary where each session key maps to a list of actions.

    Returns:
        dict: The updated actions dictionary with 'action_type' set to the final action type.
    """
    for _, actions in actions_dict.items():
        for action in actions:
            if 'level_3_action_type' in action and action['level_3_action_type']:
                action['action_type'] = action['level_3_action_type']
            elif 'level_2_action_type' in action and action['level_2_action_type']:
                action['action_type'] = action['level_2_action_type']
            else:
                action['action_type'] = action['level_1_action_type']
    return actions_dict

In [24]:
# Test with 'legislation_corporate_2'
test_session_dict = {
    "legislation_corporate_2": level_3_actions_per_session["legislation_corporate_2"]
}

updated_test_actions = determine_final_action_type_default(test_session_dict)

for action in updated_test_actions["legislation_corporate_2"]:
    print(action.get("action_type"))

topic_shifted
query_suggestion
present_suggestion
hover_over_text
minor_insert_major_semantic_diff
reject_suggestion
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
query_suggestion
present_suggestion
hover_over_text
cursor_operation
reject_suggestion
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
cursor_operation
delete_text
cursor_operation
delete_text
cursor_operation
topic_shifted
delete_text
topic_shifted
delete_text
topic_shifted


Here is an example output demonstrating how to determine the final action type using the **determine_final_action_type_default** function. This function automatically updates the **action_type** field based on the default priority logic. As shown, if a level 3 or level 2 action type is present in the output, it will be assigned as the final **action_type**.

In [25]:
def determine_final_action_type_customize(actions_dict, priority_list):
    """
    Sorts and assigns action types in the actions dictionary based on a custom priority list.

    Args:
        actions_dict (dict): Dictionary where each session key maps to a list of actions.
        priority_list (list): List of action types in descending order of priority.

    Returns:
        dict: The updated actions dictionary with 'action_type' assigned based on the custom priority list.
    """
    for _, actions in actions_dict.items():
        for action in actions:
            for priority_action in priority_list:
                if (action.get('level_1_action_type') == priority_action or
                    action.get('level_2_action_type') == priority_action or
                    action.get('level_3_action_type') == priority_action):
                    action['action_type'] = priority_action
                    break  # Assign the highest priority and stop checking
    return actions_dict

In [26]:
# Test with 'legislation_corporate_2'
test_session_dict = {
    "legislation_corporate_2": level_3_actions_per_session["legislation_corporate_2"]
}

custom_priority_list = ['major_insert_major_semantic_diff', 'minor_insert_mindless_edit']  # Paste your own choice of priority list here
updated_test_actions = determine_final_action_type_customize(test_session_dict, custom_priority_list)

for action in updated_test_actions["legislation_corporate_2"]:
    print(action.get("action_type"))

topic_shifted
query_suggestion
present_suggestion
hover_over_text
minor_insert_major_semantic_diff
reject_suggestion
topic_shifted
delete_text
major_insert_major_semantic_diff
delete_text
topic_shifted
query_suggestion
present_suggestion
hover_over_text
cursor_operation
reject_suggestion
topic_shifted
delete_text
topic_shifted
delete_text
major_insert_major_semantic_diff
delete_text
topic_shifted
delete_text
topic_shifted
delete_text
cursor_operation
delete_text
cursor_operation
delete_text
cursor_operation
topic_shifted
delete_text
major_insert_major_semantic_diff
delete_text
topic_shifted


Here is an example output demonstrating how to determine the final action type using the **determine_final_action_type_customize** function. By predefining your own priority list based on your research needs and passing it to this function, the **action_type** field will be updated accordingly. As shown, since I defined my priority list as 'major_insert_major_semantic_diff', 'minor_insert_mindless_edit', the **action_type** changes to match these values when encountered.

In [27]:
action_type_with_priority_per_session = determine_final_action_type_default(level_3_actions_per_session)

I would use the **determine_final_action_type_default** function here to save the final outputs, as it is the default method. However, feel free to try out both functions and use either one based on your preference.

### Save Prioritized Actions to a JSON File in the Current Directory

This step saves the actions dictionary with assigned and sorted priority-based action types to a JSON file. The file, named `action_type_with_priority_per_session.json`, will be created in the same directory as this script or notebook, making it easily accessible for future analysis or visualization.

By storing the prioritized actions, this step ensures the data is organized and ready for advanced analytical workflows or visual representations.

In [28]:
# Define the output file path
output_file = os.path.join(script_dir, 'action_type_with_priority_per_session_may172025.json')

# Save the prioritized actions to the JSON file
with open(output_file, 'w') as f:
    json.dump(action_type_with_priority_per_session, f, default=utils.custom_serializer)

# Confirm successful save
print(f"Prioritized action types saved to: {script_dir}")

Prioritized action types saved to: /Users/chelseashe/Thought_Toolkit


###testing ground

In [29]:
logs_by_session.keys()

dict_keys(['legislation_corporate_1', 'legislation_corporate_2', 'legislation_antitrust_1', 'legislation_antitrust_2', 'legislation_antitrust_3'])

In [30]:
user = "legislation_antitrust_1"

In [31]:
[(x['action_type'], x['action_delta'], x['action_end_writing']) for x in level_2_actions_per_session[user]]

[('topic_shifted',
  ('INSERT',
   'explain the debate about zero-price markets and antitrust laws regarding it? ',
   77,
   11),
  'what is the debate about zero-price markets and antitrust laws regarding it? '),
 ('query_suggestion',
  '',
  'what is the debate about zero-price markets and antitrust laws regarding it? '),
 ('present_suggestion',
  '',
  'what is the debate about zero-price markets and antitrust laws regarding it? '),
 ('hover_over_text',
  '',
  'what is the debate about zero-price markets and antitrust laws regarding it? '),
 ('cursor_operation',
  '',
  'what is the debate about zero-price markets and antitrust laws regarding it? '),
 ('reject_suggestion',
  '',
  'what is the debate about zero-price markets and antitrust laws regarding it? '),
 ('cursor_operation',
  '',
  'what is the debate about zero-price markets and antitrust laws regarding it? '),
 ('query_suggestion',
  '',
  'what is the debate about zero-price markets and antitrust laws regarding it? '),

In [32]:
[(x['action_end_writing']) for x in level_2_actions_per_session[user]]

['what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the debate about zero-price markets and antitrust laws regarding it? ',
 'what is the de

In [33]:
[(x['action_delta']) for x in level_2_actions_per_session[user]]

[('INSERT',
  'explain the debate about zero-price markets and antitrust laws regarding it? ',
  77,
  11),
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 ('INSERT', 'Zero-price markets', 18, 2),
 ('DELETE', 'Zero-price markets', 18, 2),
 ('INSERT', 'Anti-trust laws', 15, 2),
 '',
 '',
 '',
 '',
 ('DELETE', 'st laws', 7, 2),
 ('INSERT',
  'st laws are used to break up monopolies, a practice that the government ',
  72,
  13),
 ('DELETE', 'e that the government ', 22, 4),
 ('INSERT', ' normall', 8, 1),
 ('DELETE', 'es, a practic normall', 21, 4),
 ('INSERT', ' ', 1, 0),
 '',
 '',
 '',
 '',
 '',
 '',
 ('DELETE', ' ', 1, 0),
 ('INSERT', ' \ndefinie monopoly', 18, 2),
 '',
 '',
 '',
 '',
 '',
 ('INSERT',
  'A monopoly exists when a single company or organization dominates an industry, limiting competition and controlling prices, which can lead to unfair market conditi \ndefine anti-trust law',
  186,
  27),
 '',
 '',
 '',
 '',
 '',
 ('DELETE', 'fine a', 6, 2),
 '',
 '',
 '',
